# Module_0. Проект: игра "Угадай число!"

Предложенный автором модуля алгоритм поиска случайного числа в диапазоне от 1 до 100 можно значительно оптимизировать при помощи метода дихотомии (деления отрезка пополам) - одного из методов вычислительной математики.

В качестве источника информации могу порекомендовать книгу моего лектора и семинариста по данному предмету ( МФТИ, ФУПМ, 1994 - 1995 гг.) "Добыть" её можно, например, по ссылке: 

[В.С. Рябенький, Введение в вычислительную математику.](https://obuchalka.org/2016052589454/vvedenie-v-vichislitelnuu-matematiku-ryabenkii-v-s-2008.html)

## Суть метода заключается в следующем:

<span style="color:red"> На первом шаге в качестве предполагаемого кандидата выступает половина отрезка, на котором было сгенерировано случайное число. Если предположение больше искомого, то на следующем шаге поиск осуществляется на отрезке с началом, равным началу отрезка на предыдущем шаге, и концом, равным предположению. Если же предположение меньше искомого, то поиск на следующем шаге будет осуществляться на отрезке, началом которого является предположение, а концом - конец предыдущего шага. Предположением на следующем шаге будет выступать середина уже нового, укороченного, отрезка. С каждым шагом алгоритм будет всё ближе приближаться к искомому числу. <span>

Ниже приведена программа, которая работает по указанному алгоритму, а также проверяется, за какое в среднем количество шагов он ищет ответ для 1000 случайных чисел (варианты алгоритмов, предложенных автором модуля, я здесь повторять не стал; их можно найти в одном из файлов в репозитории):

In [6]:
import numpy as np


def game_core_v3(number):
    
    '''МОЙ ВАРИАНТ: угадываем методом дихотомии (деления отрезка пополам):
    в качестве очередного предположения выступает число, равное половине разницы
    между предыдущим и последующим. Алгоритм угадывает не более чем за 7 шагов
    (для случая, если длина отрезка равна 100). Может быть модифицирован вместе
    с генератором случайных чисел для отрезка произвольной длины.'''
    
    count = 0 # счётчик попыток
    a = 1 # нижняя граница отрезка; первоначальное значение = 1
    b = 100 # верхняя граница отрезка; первоначальное значение = 100
    predict = 0 # инициализация переменной predict - текущего предположения
    predict_old = 0 # инициализация переменной predict_old - предыдущего предположения
    
    '''далее - цикл, осуществляющий дихотомию'''
    while number != predict:
        '''очередное предположение по формуле дихотомии'''
        predict = a + (b - a + 1) // 2
        '''необходимо делать проверку при приближении к концам исходного отрезка -
        - около 1 и 100: если очередное предположение совпадает с предыдущим,
        то искомое значение равно конецу отрезка (1 или 100)'''
        if (predict, predict_old) == (2, 2):
            predict = 1
            break
        elif (predict, predict_old) == (99,99):
            predict = 100
            break 
        # если нет - то находимся пока в процессе поиска внутри очередного отрезка [a; b]
        elif number > predict:
            a = predict
            b = max(b, predict_old)
        elif number < predict:
            b = predict
            a = min(a, predict_old)
        predict_old = predict
        count += 1
    
    return(count)

        
def score_game(game_core):
    '''Это - функция автора модуля, в неё изменения не вносилось.'''
    '''Запускаем игру 1000 раз, чтобы узнать, как быстро игра угадывает число'''
    count_ls = []
    np.random.seed(1)  # фиксируем RANDOM SEED, чтобы ваш эксперимент был воспроизводим!
    random_array = np.random.randint(1,101, size=(1000))
    for number in random_array:
        count_ls.append(game_core(number))
    score = int(np.mean(count_ls))
    print(f"Ваш алгоритм угадывает число в среднем за {score} попыток")
    return(score)


score_game(game_core_v3)

Ваш алгоритм угадывает число в среднем за 5 попыток


5

Несмотря на то, что программа выдаёт ответ, что алгоритм угадывает число в среднем за 5 попыток, необходимо иметь ввиду, что гарантированное число попыток - всё-таки 7.

<span style="color:red"> <b> ВНИМАНИЕ! </b> <span>

<span style="color:blue"> Предложенный алгоритм может быть усовершенствован для случая, когда верхняя и нижняя границы числового отрезка, в котором генерируются случайные числа - задаются пользователем. Обозначим их соответственно a и b. <span>

В этом случае код программы примет вид:

In [7]:
import numpy as np


def game_core_v3(number, a, b):
    
    '''описание даётся только к новым переменным'''
    count = 0
    a_in = a # внутренняя переменная функции для нижней границы отрезка
    b_in = b # внутренняя переменная функции для верхней границы отрезка
    predict = 0 
    predict_old = 0 
    
    while number != predict:
        predict = a_in + (b_in - a_in + 1) // 2
        if (predict, predict_old) == (a + 1, a + 1):
            predict = a
            break
        elif (predict, predict_old) == (b - 1, b - 1):
            predict = b
            break  
        elif number > predict:
            a_in = predict
            b_in = max(b_in, predict_old)
        elif number < predict:
            b_in = predict
            a_in = min(a_in, predict_old)
        predict_old = predict
        count += 1
    return(count)

        
def score_game():
    '''Запускаем игру 1000 раз, чтобы узнать, как быстро игра угадывает число'''
    a = int(input('Задайте нижнюю границу отрезка: '))
    b = int(input('Задайте верхнюю границу отрезка: '))
    count_ls = []
    np.random.seed(1)
    random_array = np.random.randint(a, b + 1, size=(1000)) # случайное число генерируется на заданной отрезке
    for number in random_array:
        count_ls.append(game_core_v3(number, a, b))
    score = int(np.mean(count_ls))
    print(f"Ваш алгоритм угадывает число в среднем за {score} попыток")
    return(score)

score_game()

Задайте нижнюю границу отрезка: 1
Задайте верхнюю границу отрезка: 100
Ваш алгоритм угадывает число в среднем за 5 попыток


5

Для произвольного отрезка \[a, b\] количество гарантированных попыток равно <b><span style="color:blue"> int(math.log2(b - a)) + 1 <span> </b>.

 = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =  

_В репозитории находятся следующие файлы:_

_Module_0_only_my_variant.py - мой алгоритм поиска случайного числа от 1 до 100_

_Module_0_all_variants.py - включает не только мой алгоритмы, но и алгоритмы автора модуля_

_Module_0_advanced_variant.py - мой алгоритм поиска случайного числа от a до b_